In [3]:
# Set dataset path
data_path = r'/data/training_data/VTF/training_data_noise_extended'
extension = "*.csv"

# Hyperparameters that will alter throughout the model creations
input_size = 26  # Number of CAN signals per timestep
hidden_size = [256, 256, 512, 512, 512]
num_layers = [3, 4, 3, 4, 3]
learning_rate = [0.0001] * 5
# num of sequences in one batch
batch_size = [128] * 5
dropout_rate = [0.2] * 5
sequence_length = [800, 800, 800, 800, 1000]


# parameters of the simulation
step_size = 10 # what the overlap between the sequences should look like in the extracted dataset
output_size = 1
num_epochs = 20

num_models = 5

default_location = r"C:\my files\thesis\AI_training\Simple RNN\trained_models\iteration_7\model_"

In [4]:
# Training loops
for j in range(num_models):
    # Load dataset and DataLoader
    dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[j], step_size = step_size)
    dataloader = DataLoader(dataset, batch_size=batch_size[j], shuffle=True, num_workers= 6, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SpeedEstimatorRNN(input_size, hidden_size[j], num_layers[j], output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[j])

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx, (features, speeds) in enumerate(dataloader):
            features, speeds = features.to(device), speeds.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, speeds)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Model: {j}, Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "sequence_length": sequence_length,
        "input_size": input_size,
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "output_size": output_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs
    }, default_location + str(j) + ".pt")

Model: 0, Epoch [1/20], Loss: 40.2904
Model: 0, Epoch [2/20], Loss: 7.4543
Model: 0, Epoch [3/20], Loss: 2.1809
Model: 0, Epoch [4/20], Loss: 0.8189
Model: 0, Epoch [5/20], Loss: 0.3578
Model: 0, Epoch [6/20], Loss: 0.1766
Model: 0, Epoch [7/20], Loss: 0.0968
Model: 0, Epoch [8/20], Loss: 0.0565
Model: 0, Epoch [9/20], Loss: 0.0347
Model: 0, Epoch [10/20], Loss: 0.0241
Model: 0, Epoch [11/20], Loss: 0.0174
Model: 0, Epoch [12/20], Loss: 0.0137
Model: 0, Epoch [13/20], Loss: 0.0106
Model: 0, Epoch [14/20], Loss: 0.0105
Model: 0, Epoch [15/20], Loss: 0.0121
Model: 0, Epoch [16/20], Loss: 0.0068
Model: 0, Epoch [17/20], Loss: 0.0073
Model: 0, Epoch [18/20], Loss: 0.0086
Model: 0, Epoch [19/20], Loss: 0.0071
Model: 0, Epoch [20/20], Loss: 0.0073
Model: 1, Epoch [1/20], Loss: 43.6167
Model: 1, Epoch [2/20], Loss: 8.1757
Model: 1, Epoch [3/20], Loss: 2.2464
Model: 1, Epoch [4/20], Loss: 0.8246
Model: 1, Epoch [5/20], Loss: 0.3609
Model: 1, Epoch [6/20], Loss: 0.1792
Model: 1, Epoch [7/20], L

IndexError: list index out of range